In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories, plot_MC
from chimera_fgo.odometry import odometry

%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.express as px
colors = px.colors.qualitative.D3

In [ ]:
kitti_seq = '0027'

# if kitti_seq == '0027':
#     fgo_run = 'fgo_200m_20runs_100w'
#     fgo_blind_run = 'fgo_200m_20runs_100w_blind'
# else:
#     fgo_run = 'fgo_200m_10runs_100w'
#     fgo_blind_run = 'fgo_200m_10runs_100w_blind'
fgo_run = 'fgo_200m_20runs_100w'
fgo_blind_run = 'fgo_200m_20runs_100w_blind'
N = 1900
N_BLIND = 1900

In [ ]:
for kitti_seq in ['0018', '0027', '0028', '0034']:
    fgo_results_path = os.path.join(os.getcwd(), '..', '..', 'results', 'error_plots', kitti_seq, fgo_run)
    fgo_results_files = os.listdir(fgo_results_path)

    fgo_blind_results_path = os.path.join(os.getcwd(), '..', '..', 'results', 'error_plots', kitti_seq, fgo_blind_run)
    fgo_blind_results_files = os.listdir(fgo_blind_results_path)

    if kitti_seq == '0027':
        start_idx = 1000
        lidar_start_idx = 0
    elif kitti_seq == '0028':
        start_idx = 1550
        lidar_start_idx = 1550
    else:
        start_idx = 0
        lidar_start_idx = 0
    gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
    gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)
    data_path = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'icp')
    lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=lidar_start_idx)
    if kitti_seq == '0027':
        lidar_Rs = lidar_Rs[start_idx:]
        lidar_ts = lidar_ts[start_idx:]
        init_pose = (gt_Rs[0].copy(), gt_enu[0].copy())
        lidar_odom = [(R, t) for R, t in zip(lidar_Rs, lidar_ts)]
        lidar_positions, _ = odometry(init_pose, lidar_odom)

    n_runs = min(len(fgo_results_files), len(fgo_blind_results_files))
    fgo_blind_err_avg = np.zeros(N_BLIND)
    fgo_err_avg = np.zeros(N)
    time = np.arange(N) / 10

    fig = go.Figure()
    fig.add_vline(x=100, line_width=1, line_dash="dash", line_color="red")

    nd = 3

    for i in range(n_runs):
        fgo_results = np.load(os.path.join(fgo_results_path, fgo_results_files[i]))
        fgo_blind_results = np.load(os.path.join(fgo_blind_results_path, fgo_blind_results_files[i]))
        try:
            fgo_positions = fgo_results['fgo_positions']
        except:
            fgo_positions = fgo_results['positions']
        try:
            fgo_blind_positions = fgo_blind_results['fgo_positions']
        except:
            fgo_blind_positions = fgo_blind_results['positions']
        
        fgo_blind_err = np.linalg.norm(fgo_blind_positions[:N_BLIND,:nd] - gt_enu[:N_BLIND,:nd], axis=1)
        fgo_err = np.linalg.norm(fgo_positions[:N,:nd] - gt_enu[:N,:nd], axis=1)
        showlegend = True if i == 0 else False
        fig.add_trace(go.Scatter(x=time, y=fgo_blind_err, name='Naive FGO', line=dict(color='orange'), showlegend=showlegend))
        fig.add_trace(go.Scatter(x=time, y=fgo_err, name='SR FGO', line=dict(color='blue'), showlegend=showlegend))


    lidar_err = np.linalg.norm(lidar_positions[:N,:nd] - gt_enu[:N,:nd], axis=1)
    fgo_blind_err_avg /= n_runs
    fgo_err_avg /= n_runs

    fig.add_trace(go.Scatter(x=time, y=lidar_err, name='Odometry only', line=dict(color='green')))
    fig.update_layout(width=1200, height=500, xaxis_title='Time [s]', yaxis_title='L2 norm error [m]')
    fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=22))
    fig.update_layout(margin=go.layout.Margin(
        l=20, #left margin
        r=20, #right margin
        b=20, #bottom margin
        t=20  #top margin
    ))
    fig.show()

In [ ]:
lidar_positions[:N,:2]

In [ ]:
np.mean(fgo_blind_err[:1000])

In [ ]:
np.mean(fgo_err[:1000])